# Tutorial 2: **Notebook** - *FVF*

> 🚧 **Under Construction**

**By 
gLayout Team**

__Content creators:__ Subham Pal, Saptarshi Ghosh

__Content reviewers:__ Mehedi Saligne

___
# Tutorial Objectives

This notebook is a tutorial on-

- **LVS (Layout Versus Schematic):**  
  You will learn how to compare your physical layout with the original schematic to ensure they are functionally identical. This process helps catch connectivity or device mismatches before fabrication.

- **Extraction and Simulation:**  
  The tutorial will guide you through extracting parasitic elements from your layout, such as capacitance and resistance, to create a more accurate circuit model. You will then simulate the extracted netlist to analyze and verify the real-world performance of your design.

## **Target** **Block** : **Flipped Voltage Follwer Cell**

A **voltage follower**—also known as a unity-gain buffer or buffer amplifier—is an electronic circuit in which the output voltage precisely follows the input voltage, providing a voltage gain of one. Typically implemented using an operational amplifier (op-amp) with negative feedback, the voltage follower features extremely high input impedance and very low output impedance. This configuration allows it to isolate circuit stages, preventing the loading of the input source and enabling the circuit to drive low-impedance loads without signal degradation.

The voltage follower is fundamental in analog circuit design, ensuring signal fidelity and stability across a wide range of electronic applications. The **Flipped Voltage Follower (FVF)** is an advanced analog circuit topology derived from the conventional source follower, optimized for low-voltage, low-power applications. Unlike the standard voltage follower, the FVF employs a feedback structure that forces the input transistor to operate at a constant drain current, independent of variations in input voltage or load current. This is achieved using shunt negative feedback and ancillary biasing circuitry, resulting in improved linearity and significantly reduced output impedance compared to traditional designs.

**Key Features:**
- **Low Output Impedance:** The FVF provides much lower output impedance than conventional voltage followers, making it highly effective as a voltage buffer in demanding analog applications[9][11].
- **High Linearity:** Maintains a consistent voltage transfer characteristic across a wide range of operating conditions[5][9].
- **Large Output Current Capability:** Able to source or sink larger currents, supporting class-AB operation and driving heavier loads[4][7][10].
- **Low-Voltage Operation:** Well-suited for modern low-supply-voltage and low-power integrated circuit designs[4][5][7][10].
- **Applications:** Commonly found in output stages, current mirrors, voltage buffers, gain-boosting circuits, OTAs, filters, and VCOs[4][5][7][10][11].

The FVF is a versatile and robust building block in analog and mixed-signal circuit design, offering superior performance for buffering, level shifting, and driving loads in advanced CMOS technologies.

(a) Conventional Voltage follower (common Drain); (b) Flipped voltage follower (FVF).

![](_images/FVF.png)

```bibtex
Domala, N., Sasikala, G. Low power flipped voltage follower current mirror with improved input output impedances. Sādhanā 46, 142 (2021). https://doi.org/10.1007/s12046-021-01665-6
```

## **NetList generation and LVS**
let's go through the step by step procedure to generate LVS and DRC clean layout of a FVF cell.

In [1]:
import os
import subprocess

# Run a shell, source .bashrc, then printenv
cmd = 'bash -c "source ~/.bashrc && printenv"'
result = subprocess.run(cmd, shell=True, text=True, capture_output=True)
env_vars = {}
for line in result.stdout.splitlines():
    if '=' in line:
        key, value = line.split('=', 1)
        env_vars[key] = value

# Now, update os.environ with these
os.environ.update(env_vars)

In [2]:
from glayout import MappedPDK, sky130 , gf180
#from gdsfactory.cell import cell
from gdsfactory import Component
from gdsfactory.components import text_freetype, rectangle

In [3]:
from glayout import nmos, pmos
from glayout import via_stack
from glayout import rename_ports_by_orientation
from glayout import tapring

In [4]:
from glayout.util.comp_utils import evaluate_bbox, prec_center, prec_ref_center, align_comp_to_port
from glayout.util.port_utils import add_ports_perimeter,print_ports
from glayout.util.snap_to_grid import component_snap_to_grid
from glayout.spice.netlist import Netlist

In [5]:
from glayout.routing.straight_route import straight_route
from glayout.routing.c_route import c_route
from glayout.routing.L_route import L_route

FVF has two fets as shown in the schematic. We call M1 as input fet and M2 as feedback fet. Lets define arguments for the FETs

### 2. Basic Usage of the GLayout Framework
Each generator is a Python function that takes a `MappedPDK` object as a parameter and generates a DRC clean layout for the given PDK. The generator may also accept a set of optional layout parameters such as the width or length of a MOSFET. All parameters are normal Python function arguments.

The generator returns a `GDSFactory.Component` object that can be written to a `.gds` file and viewed using a tool such as Klayout. In this example, the `gdstk` library is used to convert the `.gds` file to an SVG image for viewing.

The pre-PEX SPICE netlist for the component can be viewed using `component.info['netlist'].generate_netlist()`.

In the following example the FET generator `glayout.primitives.fet` is imported and run with both the [Skywater 130](https://skywater-pdk.readthedocs.io/en/main/) and [GF180](https://gf180mcu-pdk.readthedocs.io/en/latest/) PDKs.

#### Demonstration of Basic Layout / Netlist Generation in SKY130 & GF180

In [6]:
import gdstk
import svgutils.transform as sg
import IPython.display
from IPython.display import clear_output
import ipywidgets as widgets

# Used to display the results in a grid (notebook only)
left = widgets.Output()
leftSPICE = widgets.Output()
right = widgets.Output()
rightSPICE = widgets.Output()
hide = widgets.Output()

grid = widgets.GridspecLayout(1, 4)
grid[0, 0] = left
grid[0, 1] = leftSPICE
grid[0, 2] = right
grid[0, 3] = rightSPICE
display(grid)

def display_gds(gds_file, scale = 3):
  # Generate an SVG image
  top_level_cell = gdstk.read_gds(gds_file).top_level()[0]
  top_level_cell.write_svg('../../out.svg')

  # Scale the image for displaying
  fig = sg.fromfile('../../out.svg')
  fig.set_size((str(float(fig.width) * scale), str(float(fig.height) * scale)))
  fig.save('../../out.svg')

  # Display the image
  IPython.display.display(IPython.display.SVG('../../out.svg'))

def display_component(component, scale = 3):
  # Save to a GDS file
  with hide:
    component.write_gds("../../out.gds")

  display_gds('../../out.gds', scale)

with hide:
  # Generate the sky130 component
  component_sky130 = nmos(pdk = sky130, fingers=5)
  # Generate the gf180 component
  component_gf180 = nmos(pdk = gf180, fingers=5,with_dnwell=False)

# Display the components' GDS and SPICE netlists
with left:
  print('Skywater 130nm N-MOSFET (fingers = 5)')
  display_component(component_sky130, scale=2)
with leftSPICE:
  print('Skywater 130nm SPICE Netlist')
  print(component_sky130.info['netlist'].generate_netlist())

with right:
  print('GF 180nm N-MOSFET (fingers = 5)')
  display_component(component_gf180, scale=2)
with rightSPICE:
  print('GF 180nm SPICE Netlist')
  print(component_gf180.info['netlist'].generate_netlist())

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

### This part is just to import the file you created from Tutorial 1

In [7]:
import sys
import os
from pathlib import Path
sys.path.append(os.path.abspath("../../FVF"))

from my_FVF import flipped_voltage_follower,add_fvf_labels

### Creating A Netlist

In [8]:
# This is a different way of creating netlist compared to what we already have in the repo.
# Instead of giving the low level components as input, you append them to the top_level.info in component function.
# Then you can access them as showed here.

def fvf_netlist(fvf_in: Component) -> Netlist:

    fet_1 = fvf_in.info["fet_1"]  
    fet_2 = fvf_in.info["fet_2"]
    
    netlist = Netlist(circuit_name='FLIPPED_VOLTAGE_FOLLOWER', nodes=['VIN', 'VBULK', 'VOUT', 'Ib'])
    
    netlist.connect_netlist(fet_1.info['netlist'], [('D', 'Ib'), ('G', 'VIN'), ('S', 'VOUT'), ('B', 'VBULK')])
    netlist.connect_netlist(fet_2.info['netlist'], [('D', 'VOUT'), ('G', 'Ib'), ('S', 'VBULK'), ('B', 'VBULK')])
    
    fvf_in.info['netlist'] = netlist
    
    return fvf_in

my_fvf = fvf_netlist(flipped_voltage_follower(gf180, width=(4,2.75), length=(2,1)))
my_fvf  = add_fvf_labels(my_fvf, gf180)
my_fvf .name = "FVF"
#my_fvf.write_gds('out_FVF.gds')
my_fvf.show()
print(my_fvf.info['netlist'].generate_netlist())

2025-07-03 13:34:16.315 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.3.3", "klayout_version": "0.30.2", "type": "reload", "file": "/tmp/gdsfactory/FVF.gds"}


.subckt NMOS D G S B l=2 w=4 m=1 dm=1 
XMAIN   D G S B nfet_03v3 l={l} w={w} m={m}
XDUMMY1 B B B B nfet_03v3 l={l} w={w} m={dm}
XDUMMY2 B B B B nfet_03v3 l={l} w={w} m={dm}
.ends NMOS

.subckt NMOS_1 D G S B l=1 w=2.75 m=1 dm=1 
XMAIN   D G S B nfet_03v3 l={l} w={w} m={m}
XDUMMY1 B B B B nfet_03v3 l={l} w={w} m={dm}
XDUMMY2 B B B B nfet_03v3 l={l} w={w} m={dm}
.ends NMOS_1

.subckt FLIPPED_VOLTAGE_FOLLOWER VIN VBULK VOUT Ib
X0 Ib VIN VOUT VBULK NMOS l=2 w=4 m=0.5 dm=1
X1 VOUT Ib VBULK VBULK NMOS_1 l=1 w=2.75 m=0.5 dm=1
.ends FLIPPED_VOLTAGE_FOLLOWER


### Run LVS
 LVS(Layout Versus Schematic) is an automated process that compares the extracted netlist of the physical layout (how the transistors and wires are actually drawn on the silicon) against the original, intended netlist from the schematic (the circuit's functional design). Its primary goal is to ensure that the layout precisely matches the schematic, catching any discrepancies like shorts, opens, missing components, or incorrect connections. `Netgen` is the tool we use for LVS here.

In [9]:
# This code block is inserted to delete previously stored intermediate files, if any
%cd ../../FVF/
!pwd
import glob
extensions = [
            "*.res.ext",
            "*.lvs.rpt",
            "*_lvs.rpt",
            "*.nodes",
            "*.sim",
            "*.pex.spice",
            "*_pex.spice"
            ]
files_to_delete = []
for ext in extensions:
    files_to_delete.extend(glob.glob(ext))
    
# Delete the files
for file_path in files_to_delete:
    try:
        os.remove(file_path)
        print(f"Deleted: {file_path}")
    except OSError as e:
        print(f"Error deleting {file_path}: {e}")

/foss/designs/all_last2/FVF
/foss/designs/all_last2/FVF


In [10]:
fvf = fvf_netlist(add_fvf_labels(flipped_voltage_follower(gf180,width=(4,2.75),length=(2,1)),gf180))
fvf.name = "fvf"
fvf_gds = fvf.write_gds("fvf.gds")
#display_gds(fvf_gds)
fvf.show()
netgen_lvs_result = gf180.lvs_netgen(fvf, fvf.name)

2025-07-03 13:34:20.584 | INFO     | gdsfactory.component:_write_library:1851 - Wrote to 'fvf.gds'
2025-07-03 13:34:20.594 | INFO     | gdsfactory.klive:show:55 - Message from klive: {"version": "0.3.3", "klayout_version": "0.30.2", "type": "reload", "file": "/tmp/gdsfactory/fvf.gds"}
2025-07-03 13:34:20.600 | INFO     | gdsfactory.component:_write_library:1851 - Wrote to '/tmp/tmpxbjq75xn/fvf.gds'


using user specified pdk_root, will search for required files in the specified directory

Magic 8.3 revision 528 - Compiled on Wed Jun 18 09:45:25 PM CEST 2025.
Starting magic under Tcl interpreter
Using the terminal as the console.
Using NULL graphics device.
Processing system .magicrc file
Sourcing design .magicrc for technology gf180mcuD ...
10 Magic internal units = 1 Lambda
Input style import: scaleFactor=10, multiplier=2
The following types are not handled by extraction and will be treated as non-electrical types:
    obsactive mvobsactive filldiff fillpoly m1hole obsm1 fillm1 obsv1 m2hole obsm2 fillm2 obsv2 m3hole obsm3 fillm3 m4hole obsm4 fillm4 m5hole obsm5 fillm5 glass fillblock lvstext obscomment 
Scaled tech values by 10 / 1 to match internal grid scaling
Loading gf180mcuD Device Generator Menu ...
Using technology "gf180mcuD", version 1.0.525-0-gf2e289d
Library written using GDS-II Release 6.0
Library name: library
Reading "fvf".
Extracting fvf into fvf.ext:
fvf: 4 warning

## Extraction and Post-Pex Simulation

In [11]:
#At first, we need to properly set PDKPATH and PDK_ROOT 

print("Making sure your environment varriables are still correctly set")
print(" I am using PDK in: ",os.environ['PDK_ROOT'],"\n PDK name: ",os.environ['PDK'],"\n The PDK files are at: ",os.environ['PDKPATH'])

Making sure your environment varriables are still correctly set
 I am using PDK in:  /foss/pdks 
 PDK name:  gf180mcuD 
 The PDK files are at:  /foss/pdks/gf180mcuD


Sometimes, When you donot excecute the Jupyter Cells Sqencally, It might cause errors. In that case, you can set it right by re-excecuting the top cell. No needed if already set.

In [12]:
# import os
# import subprocess

# # Run a shell, source .bashrc, then printenv
# cmd = 'bash -c "source ~/.bashrc && printenv"'
# result = subprocess.run(cmd, shell=True, text=True, capture_output=True)
# env_vars = {}
# for line in result.stdout.splitlines():
#     if '=' in line:
#         key, value = line.split('=', 1)
#         env_vars[key] = value

# # Now, update os.environ with these
# os.environ.update(env_vars)

In [13]:
# import os
# os.environ["PATH"] = os.environ["TOOLS"]+'/bin:'+ os.environ["PATH"]
# # To "unset" (delete) PDKPATH if it exists
# if 'PDKPATH' in os.environ:
#     del os.environ['PDKPATH']
#     print("PDKPATH has been unset.")
# else:
#     print("PDKPATH was not set.")

# # To "unset" (delete) PDK_ROOT if it exists
# if 'PDK_ROOT' in os.environ:
#     del os.environ['PDK_ROOT']
#     print("PDK_ROOT has been unset.")
# else:
#     print("PDK_ROOT was not set.")

# # --- Now, set the correct one ---
# correct_pdk_path = "/foss/pdks/sky130A" # <-- Replace with the ACTUAL path to your Sky130A PDK root
# os.environ['PDKPATH'] = correct_pdk_path
# print(f"PDKPATH set to: {os.environ['PDKPATH']}")
# %env PDK = "sky130A"
# !echo $PDK_ROOT
# !echo $PDK
# !echo $PDKPATH

In [14]:
magic_file = Path(os.environ['PDKPATH']) / "libs.tech" / "magic" / f"{os.environ['PDK']}.magicrc"
magic_file

PosixPath('/foss/pdks/gf180mcuD/libs.tech/magic/gf180mcuD.magicrc')

### Parasitic Extraction
Two files are needed for parasitic extraction. One is foundry provided sky130A.magicrc. Other one is our bash script run_pex.sh which will call the `Magic` terminal and automate the task

In [ ]:
with tempfile.TemporaryDirectory() as temp_dir:
            temp_dir_path = Path(temp_dir).resolve()
            self.pdk_files['temp_dir'] = temp_dir_path
            print("using user specified pdk_root, will search for required files in the specified directory")
            self.pdk_files['pdk_root'] = pdk_root 
            
            lvsmag_path = temp_dir_path / f"{design_name}_lvsmag.spice"
            pex_path = temp_dir_path / f"{design_name}_pex.spice"
            sim_path = temp_dir_path / f"{design_name}_sim.spice"
            spice_path = temp_dir_path / f"{design_name}.spice"
            netlist_from_comp = temp_dir_path / f"{design_name}.cdl"
            gds_path = temp_dir_path / f"{design_name}.gds"
            report_path = temp_dir_path / f"{design_name}_lvs.rpt"

            layout.write_gds(str(gds_path))

    

In [ ]:
magic_script_content = f"""#!/bin/bash

# Usage: ./run_pex.sh layout.gds layout_cell_name

GDS_FILE=$1
LAYOUT_CELL=$2

magic -rcfile ./sky130A.magicrc -noconsole -dnull << EOF
gds read $GDS_FILE
flatten $LAYOUT_CELL
load $LAYOUT_CELL
select top cell
extract do local
extract all
ext2sim labels on
ext2sim
extresist tolerance 10
extresist
ext2spice lvs
ext2spice cthresh 0
ext2spice extresist on
ext2spice -o ${LAYOUT_CELL}_pex.spice
exit
EOF
"""

In [ ]:
run_pex_string = """#!/bin/bash

# Usage: ./run_pex.sh layout.gds layout_cell_name

GDS_FILE=$1
LAYOUT_CELL=$2

magic -rcfile ./sky130A.magicrc -noconsole -dnull << EOF
gds read $GDS_FILE
flatten $LAYOUT_CELL
load $LAYOUT_CELL
select top cell
extract do local
extract all
ext2sim labels on
ext2sim
extresist tolerance 10
extresist
ext2spice lvs
ext2spice cthresh 0
ext2spice extresist on
ext2spice -o ${LAYOUT_CELL}_pex.spice
exit
EOF
"""

sky130A_magicrc_string = """puts stdout "Sourcing design .magicrc for technology sky130A ..."

# Put grid on 0.005 pitch.  This is important, as some commands don't
# rescale the grid automatically (such as lef read?).

set scalefac [tech lambda]
if {[lindex $scalefac 1] < 2} {
    scalegrid 1 2
}

# drc off
drc euclidean on
# Change this to a fixed number for repeatable behavior with GDS writes
# e.g., "random seed 12345"
catch {random seed}

# Turn off the scale option on ext2spice or else it conflicts with the
# scale in the model files.
ext2spice scale off

# Allow override of PDK path from environment variable PDKPATH
if {[catch {set PDKPATH $env(PDKPATH)}]} {
    set PDKPATH $env(PDK_ROOT)/sky130A
}

# loading technology
tech load $PDKPATH/libs.tech/magic/sky130A.tech

# load device generator
source $PDKPATH/libs.tech/magic/sky130A.tcl

# load bind keys (optional)
# source $PDKPATH/libs.tech/magic/sky130A-BindKeys

# set units to lambda grid
snap lambda

# set sky130 standard power, ground, and substrate names
set VDD VPWR
set GND VGND
set SUB VSUBS

# Allow override of type of magic library views used, "mag" or "maglef",
# from environment variable MAGTYPE

if {[catch {set MAGTYPE $env(MAGTYPE)}]} {
   set MAGTYPE mag
}

# add path to reference cells
if {[file isdir ${PDKPATH}/libs.ref/${MAGTYPE}]} {
    addpath ${PDKPATH}/libs.ref/${MAGTYPE}/sky130_fd_pr
    addpath ${PDKPATH}/libs.ref/${MAGTYPE}/sky130_fd_io
    addpath ${PDKPATH}/libs.ref/${MAGTYPE}/sky130_fd_sc_hd
    addpath ${PDKPATH}/libs.ref/${MAGTYPE}/sky130_fd_sc_hdll
    addpath ${PDKPATH}/libs.ref/${MAGTYPE}/sky130_fd_sc_hs
    addpath ${PDKPATH}/libs.ref/${MAGTYPE}/sky130_fd_sc_hvl
    addpath ${PDKPATH}/libs.ref/${MAGTYPE}/sky130_fd_sc_lp
    addpath ${PDKPATH}/libs.ref/${MAGTYPE}/sky130_fd_sc_ls
    addpath ${PDKPATH}/libs.ref/${MAGTYPE}/sky130_fd_sc_ms
    addpath ${PDKPATH}/libs.ref/${MAGTYPE}/sky130_osu_sc
    addpath ${PDKPATH}/libs.ref/${MAGTYPE}/sky130_osu_sc_t18
    addpath ${PDKPATH}/libs.ref/${MAGTYPE}/sky130_ml_xx_hd
    addpath ${PDKPATH}/libs.ref/${MAGTYPE}/sky130_sram_macros
} else {
    addpath ${PDKPATH}/libs.ref/sky130_fd_pr/${MAGTYPE}
    addpath ${PDKPATH}/libs.ref/sky130_fd_io/${MAGTYPE}
    addpath ${PDKPATH}/libs.ref/sky130_fd_sc_hd/${MAGTYPE}
    addpath ${PDKPATH}/libs.ref/sky130_fd_sc_hdll/${MAGTYPE}
    addpath ${PDKPATH}/libs.ref/sky130_fd_sc_hs/${MAGTYPE}
    addpath ${PDKPATH}/libs.ref/sky130_fd_sc_hvl/${MAGTYPE}
    addpath ${PDKPATH}/libs.ref/sky130_fd_sc_lp/${MAGTYPE}
    addpath ${PDKPATH}/libs.ref/sky130_fd_sc_ls/${MAGTYPE}
    addpath ${PDKPATH}/libs.ref/sky130_fd_sc_ms/${MAGTYPE}
    addpath ${PDKPATH}/libs.ref/sky130_osu_sc/${MAGTYPE}
    addpath ${PDKPATH}/libs.ref/sky130_osu_sc_t18/${MAGTYPE}
    addpath ${PDKPATH}/libs.ref/sky130_ml_xx_hd/${MAGTYPE}
    addpath ${PDKPATH}/libs.ref/sky130_sram_macros/${MAGTYPE}
}

# add path to GDS cells

# add path to IP from catalog.  This procedure defined in the PDK script.
catch {magic::query_mylib_ip}
# add path to local IP from user design space.  Defined in the PDK script.
catch {magic::query_my_projects}
"""


directory = "./"

# Save files
file_name_1 = "run_pex.sh"
file_name_2 = "sky130A.magicrc"
# Delete the file if it already exists
if os.path.exists(file_name_1):
    os.remove(file_name_1)
    print(f"Existing file '{file_name_1}' deleted.")

# Create a new empty file
with open(file_name_1, 'x') as f:
    pass  # This creates an empty file or truncates an existing one
print(f"Empty file '{file_name_1}' created.")

if os.path.exists(file_name_2):
    os.remove(file_name_2)
    print(f"Existing file '{file_name_2}' deleted.")

with open(file_name_2, 'w') as f:
    pass  # This creates an empty file or truncates an existing one
print(f"Empty file '{file_name_2}' created.")


with open(directory + "run_pex.sh", "w") as file:
    file.write(run_pex_string)
    print(f"Wrote to '{file_name_1}'.")
with open(directory + "sky130A.magicrc", "w") as file:
    file.write(sky130A_magicrc_string)
    print(f"Wrote to '{file_name_2}'.")
pex_spice_path = "fvf_pex.spice"
if os.path.exists(pex_spice_path):
    os.remove(pex_spice_path)

#run the bash script
!chmod +x ./run_pex.sh
proc=subprocess.run(["./run_pex.sh", "./fvf.gds", fvf.name], check=True, capture_output=True, text=True)
for line in proc.stdout.splitlines():
    print(line)

### Post-PEX simulation
We do `.op` analysis and `.ac` anaysis on our fvf block.

#### `.op` analysis - find out the DC voltage at output node

In [ ]:
fvf_op_tb_string="""* FVF Output Impedance Testbench
.temp 25
.param vcm = 1.3
.param ib = 10u

************* Power Supplies *************
Vsupply VDD GND 1.8
V0 vb GND 0

************* Input Bias (DC bias at IN) *************
VIN vin GND {vcm}  ; Mid-supply bias (renamed from 'IN' to 'VIN' to avoid conflict if 'IN' is a global node or subckt port)
Ibias VDD ib {ib}

************* DUT: FVF Subcircuit *************
**Import SKY130 lib
.lib /foss/pdks/sky130A/libs.tech/ngspice/sky130.lib.spice tt

** Import fvf subcircuit
.include fvf_pex.spice
* Adjust node order if needed based on your fvf_pex.spice subcircuit definition.
* Assuming fvf subcircuit takes (vb, output_node, input_node, ibias_node)
XDUT vb vin FVF_OUT ib fvf

************* Analysis *************
* Operating point (for sanity check)
.op ; Uncomment this to run DC operating point analysis and check node voltages

.GLOBAL VDD 
.GLOBAL GND

.end
"""
file_name = "fvf_op_tb.sp"
# Delete the file if it already exists
if os.path.exists(file_name):
    os.remove(file_name)
    print(f"Existing file '{file_name}' deleted.")

# Create a new empty file
with open(file_name, 'x') as f:
    pass  # This creates an empty file or truncates an existing one
print(f"Empty file '{file_name}' created.")
with open(directory + "fvf_op_tb.sp", "w") as file:
    file.write(fvf_op_tb_string)

In [ ]:
# run ngspice in batch mode
!ngspice -b fvf_op_tb.sp

If you did not do any changes in testbench, the voltage at output node FVF_OUT should be around 0.45V

#### `.ac` analysis - find out the output impedance

In [ ]:
fvf_zo_tb_string="""* FVF Output Impedance Testbench
.temp 25
.param vcm = 1.3
.param ib = 10u

************* Power Supplies *************
Vsupply VDD GND 1.8
V0 vb GND 0

************* Input Bias (DC bias at IN) *************
VIN vin GND {vcm}  ; Mid-supply bias (renamed from 'IN' to 'VIN' to avoid conflict if 'IN' is a global node or subckt port)
Ibias VDD ib {ib}

************* AC Test Source for Output Impedance Measurement *************
*Insert an AC current source at the output

I_TEST GND FVF_OUT AC 1u ; AC 1 magnitude, 0 DC offset, in series with FVF output

************* DUT: FVF Subcircuit *************
**Import SKY130 lib
.lib /foss/pdks/sky130A/libs.tech/ngspice/sky130.lib.spice tt

** Import fvf subcircuit
.include fvf_pex.spice
* Adjust node order if needed based on your fvf_pex.spice subcircuit definition.
* Assuming fvf subcircuit takes (vb, output_node, input_node, ibias_node)
XDUT vb vin FVF_OUT ib fvf

************* AC analysis **************
* AC sweep to compute output impedance magnitude and phase
.ac dec 10 10 10G
************* Control Output *************
* Calculate Output impedance: Zout = V(FVF_OUT) / 1u
.plot AC MAG(V(FVF_OUT))/1e-6 ; Plots the magnitude of the calculated output impedance

.GLOBAL VDD 
.GLOBAL GND

.end
"""
file_name = "fvf_zo_tb.sp"
# Delete the file if it already exists
if os.path.exists(file_name):
    os.remove(file_name)
    print(f"Existing file '{file_name}' deleted.")

# Create a new empty file
with open(file_name, 'x') as f:
    pass  # This creates an empty file or truncates an existing one
print(f"Empty file '{file_name}' created.")
with open(directory + "fvf_zo_tb.sp", "w") as file:
    file.write(fvf_zo_tb_string)

In [ ]:
# run ngspice in batch mode
!ngspice -b fvf_zo_tb.sp